# Part 3: Peer-to-peer Message Behaviour Data Analysis

---

### Install Python packages (pip only)

In [1]:
#e.g., %pip install networkx

### Import Python packages

In [2]:
import networkx as nx
import pandas as pd
import datetime
import csv
import operator
import numpy as np
import matplotlib.pyplot as plt

---

### Task 1 of 2

Examine the file "p2p_msg_cmt224.csv" which represents messaging behaviour between users on a messaging platform. Each row has four columns, representing a single event where a person (person_a) messaged another person (person_b) on some date (date) at some time of day (time). From this, answer the following questions:

##### Q1. Build a suitable network to represent social connections based on the messaging behaviour that took place up to and including the first day of May. In doing so, assume that one or more messages from one person to another represents a MUTUAL underlying social connection (i.e., regardless of whether person_a messaged person_b, person_b messaged person_a, or both at some point). 

In [3]:
# Load the dataset into a pandas DataFrame
df = pd.read_csv('p2p_msg_cmt224.csv')

# Convert the date column to a datetime object
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

# Filter the dataset to only include events that occurred on or before May 1, 2004
df = df[df['date'] <= '2004-05-01']

# Create an empty undirected graph
G1 = nx.Graph()

# Iterate over the filtered rows of the dataset and add edges to the graph
for _, row in df.iterrows():
    # Check if the edge already exists
    if G1.has_edge(row['person_a'], row['person_b']):
        # Increment the weight by 1
        G1[row['person_a']][row['person_b']]['weight'] += 1
    else:
        # Add a new edge with a weight of 1
        G1.add_edge(row['person_a'], row['person_b'], weight=1)

# Return the resulting graph
print(G1)

Graph with 544 nodes and 1839 edges


##### Q2. Build another suitable network to represent social connections based on ALL message behaviour in the dataset. In doing so, assume that one or messages from one person to another represents a MUTUAL underlying social connection (i.e., regardless of whether person_a messaged person_b, person_b messaged person_a, or both at some point).

##### Can the social phenomenon, ‘Triadic Closure’, be supported for the common nodes that exist in both the network created from data up to and including the first day of May (i.e., from Task 1, Q1) and the network built from all message behaviour?

In [4]:
# Load the dataset into a pandas DataFrame
csv = pd.read_csv('p2p_msg_cmt224.csv')

# Create an empty undirected graph
G2 = nx.Graph()

# Iterate over the filtered rows of the dataset and add edges to the graph
for _, row in csv.iterrows():
    # Check if the edge already exists
    if G2.has_edge(row['person_a'], row['person_b']):
        # Increment the weight by 1
        G2[row['person_a']][row['person_b']]['weight'] += 1
    else:
        # Add a new edge with a weight of 1
        G2.add_edge(row['person_a'], row['person_b'], weight=1)

# Calculate the transitivity for both networks
G1_transitivity = nx.transitivity(G1)
G2_transitivity = nx.transitivity(G2.subgraph(G1.nodes()))

# Calculate the clustering coefficient for both networks
G1_clustering_coefficient = nx.average_clustering(G1)
G2_clustering_coefficient = nx.average_clustering(G2.subgraph(G1.nodes()))

# Print the result to 2 decimal places unless it is less than 0.01
print(f"Transitivity of the messages up to May graph: {G1_transitivity:.2f}" if G1_transitivity >= 0.01 else f"Transitivity of the messages up to May graph: {G1_transitivity:.5f}")
print(f"Transitivity of all the messages graph: {G2_transitivity:.2f}" if G2_transitivity >= 0.01 else f"Transitivity of all the messages graph: {G2_transitivity:.5f}")
print(f"Clustering coefficient of the messages up to May graph: {G1_clustering_coefficient:.2f}" if G1_clustering_coefficient >= 0.01 else f"Clustering coefficient of the messages up to May graph: {G1_clustering_coefficient:.5f}")
print(f"Clustering coefficient of all the messages graph: {G2_clustering_coefficient:.2f}" if G2_clustering_coefficient >= 0.01 else f"Clustering coefficient of all the messages graph: {G2_clustering_coefficient:.5f}")

Transitivity of the messages up to May graph: 0.05
Transitivity of all the messages graph: 0.09
Clustering coefficient of the messages up to May graph: 0.07
Clustering coefficient of all the messages graph: 0.13


##### Q3. Using the largest connected component of the network constructed from all data in Task 1, Q2, what is the mean, median and standard deviation of the MAXIMUM degree of separation between an individual and all others?

In [5]:
# Get the largest connected component of the network
largest_cc = max(nx.connected_components(G2), key=len)
G_largest_cc = G2.subgraph(largest_cc)

# Compute the maximum degree of separation for each node
max_distances = []
for node in G_largest_cc.nodes():
    distances = nx.shortest_path_length(G_largest_cc, node)
    max_distance = max(distances.values())
    max_distances.append(max_distance)

# Filter out nodes with undefined maximum degree of separation
max_distances = [distance for distance in max_distances if not np.isnan(distance)]

# Compute the statistics
mean_max_distance = np.mean(max_distances)
median_max_distance = np.median(max_distances)
std_max_distance = np.std(max_distances)

# Print the result to 2 decimal places unless it is less than 0.01
print(f"Mean maximum degree of separation: {mean_max_distance:.2f}" if mean_max_distance >= 0.01 else f"Mean maximum degree of separation: {mean_max_distance:.5f}")
print(f"Median maximum degree of separation: {median_max_distance:.2f}" if median_max_distance >= 0.01 else f"Median maximum degree of separation: {median_max_distance:.5f}")
print(f"Standard deviation of maximum degree of separation: {std_max_distance:.2f}" if std_max_distance >= 0.01 else f"Standard deviation of maximum degree of separation: {std_max_distance:.5f}")

Mean maximum degree of separation: 5.53
Median maximum degree of separation: 6.00
Standard deviation of maximum degree of separation: 0.57


##### Q4. What hypothetical, non-existent edges would need to be added to the network such that a message could pass along a path from any node to any other? In doing so, aim to minimise the number of edges that would be needed as well as the longest shortest path in the network as a result.

In [6]:
# Create a graph from the edges in the data
mygraph = nx.from_pandas_edgelist(df, source='person_a', target='person_b', create_using=nx.Graph)
print('Initial number of edges:', mygraph.number_of_edges())

# Find the connected components of the graph
connected_comp = sorted(nx.connected_components(mygraph))

# If the graph is already fully connected, print a message and exit
if len(connected_comp) == 1:
    print('The graph is already fully connected.')
    print('Number of edges:', mygraph.number_of_edges())
    if nx.is_connected(mygraph):
        print('Longest shortest path:', nx.diameter(mygraph))
    exit()

# Find the nodes with highest degree centrality
highest_degree = sorted([(node[0], node[1]) for node in nx.degree_centrality(mygraph).items()], key=operator.itemgetter(1), reverse=True)

# Add edges to connect the nodes with highest degree centrality to each other
while not nx.is_connected(mygraph):
    for i in range(1, len(highest_degree), 10):
        for j in range(i + 1, min(i + 7, len(highest_degree))):
            node1 = highest_degree[i][0]
            node2 = highest_degree[j][0]
            if not mygraph.has_edge(node1, node2):
                mygraph.add_edge(node1, node2)
    
print('All nodes are now connected.')
print('Number of edges:', mygraph.number_of_edges())
print('Longest shortest path:', nx.diameter(mygraph))

Initial number of edges: 1839
All nodes are now connected.
Number of edges: 2159
Longest shortest path: 7


### Task 2 of 2 

Using the largest connected component of the network constructed from all data in Task 1, Q2, assume the role of an outsider with complete visibility of the network that now wishes to spread a hypothetical message such that everyone in the component would know the information it contained as quickly as possible. 

Assume that messages will now spread in sequential timesteps using the following mechanism. If an individual is told the message at timestep 𝑡, the individual will forward the message to all of their direct connections at timestep 𝑡+1. Individuals can therefore be told the message more than once. From this, answer the following questions:

##### Q1. If you could only select 1 individual to tell at timestep 0, what set of nodes could you select from which would result in the message being received by everyone in the fewest timesteps as possible and what would the number of timesteps be?

In [7]:
# Find the largest connected component of the graph
largest_cc = max(nx.connected_components(mygraph), key=len)
cc_graph = mygraph.subgraph(largest_cc)

# Compute the betweenness centrality of the nodes in the largest connected component
betweenness = nx.betweenness_centrality(cc_graph)

# Find the node with the highest betweenness centrality
starting_node = max(betweenness, key=betweenness.get)

# Print the results
print(f"The optimal node to start the message spreading is node {starting_node}.")
print(f"The number of timesteps required to reach everyone is {nx.diameter(cc_graph)}.")

The optimal node to start the message spreading is node 41.
The number of timesteps required to reach everyone is 7.


##### Q2. If you had to select any 5 individuals to tell at timestep 0, what is one example set of individuals that would result in the message being received by everyone in fewer timesteps than the single individual selection in Q1? In determining your answer, use one or more appropriate network connectivity measures for each node, rather than an exhaustive search through every combination of nodes in the network.

In [8]:
# Compute centrality measures for all nodes
degree_centrality = nx.degree_centrality(cc_graph)
betweenness_centrality = nx.betweenness_centrality(cc_graph)

# Select the top 5 nodes based on degree centrality and betweenness centrality
top_degree = sorted(degree_centrality.items(), key=operator.itemgetter(1), reverse=True)[:5]
top_betweenness = sorted(betweenness_centrality.items(), key=operator.itemgetter(1), reverse=True)[:5]

# Compute the union of the sets of nodes selected based on degree centrality and betweenness centrality
top_nodes = set([node[0] for node in top_degree] + [node[0] for node in top_betweenness])

# Simulate the spread of the message from the selected nodes and find the number of timesteps
infected_nodes = set(top_nodes)
timesteps = 0
while len(infected_nodes) < cc_graph.number_of_nodes():
    timesteps += 1
    new_infected_nodes = set()
    for node in infected_nodes:
        neighbors = cc_graph.neighbors(node)
        for neighbor in neighbors:
            if neighbor not in infected_nodes:
                new_infected_nodes.add(neighbor)
    infected_nodes.update(new_infected_nodes)

# Print the selected nodes and the number of timesteps required to reach all nodes
print('Selected nodes:', top_nodes)
print('Number of timesteps:', timesteps)

Selected nodes: {321, 103, 41, 9, 176}
Number of timesteps: 4
